# Computing features

* Relative band power (Ratios)
    * Power spectral density (PSD): represents the power distribution of EEG series in the frequency domain is used to evaluate the abnormalities of brain.
    
* Envelop spectrum of each bandT
    * Envelope analysis: the sub-bands are demodulated with Hilbert transform (HT), then envelope spectrum at each band is calculated. 
    
* Spectral connectivity (Coherence)
    * Coherence: quantifies the association between pairs of signals as a function of frequency and has been shown to be useful for measuring changes in EEG topography. Simple waveforms: transformed the frequency-filtered time-crops using the Hilbert Transform to extract the signal phase.
    
* Psi
    * The phase Slope Index (PSI) estimates the causal structure between any two source activities. I

In [1]:
band_bounds = {'theta' : [4, 8],'alpha': [8, 13],
               'beta': [13, 30],'gamma': [30, 45]}

dict_regions = {'prefrontal':['fp1','fp2'],
                   'frontal':['f7','f3','f4','fz','f8'],
                   'central':['t3','c3','cz','c4','t4'], # central and temporal
                   'parietal':['t5','p3','pz','p4','t6'],
                   'occipital':['o1','o2']}

In [2]:
!pip3 install mne

In [3]:
import mne
from os import mkdir
from os.path import exists, join
import itertools
from functools import partial
import pandas as pd
import numpy as np
from scipy import signal
from scipy.signal import hilbert
from scipy.stats import pearsonr
from scipy.integrate import simps
from tqdm import tqdm
from mne.connectivity import spectral_connectivity, phase_slope_index

# define col name
def get_col_name(method, band, ch_1, ch_2=None):
    band_name = 'nofilt' if band is None else band
    s = method + '_' + band_name + '_' + ch_1
    if ch_2:
        s += '_' + ch_2
    return s

# define file path
def get_feature_path(method_name, path):
    return join(path, method_name.replace('-', '_') + '.csv')

# Filter for each band
def get_filter(sfreq=125., band='alpha'):

    f_low_lb = band_bounds[band][0] - 1
    f_low_ub = band_bounds[band][0]
    f_high_lb = band_bounds[band][1]
    f_high_ub = band_bounds[band][1] + 1

    nyq = sfreq / 2.  # the Nyquist frequency is half our sample rate

    freq = [0., f_low_lb, f_low_ub, f_high_lb, f_high_ub, nyq]
    gain = [0, 0, 1, 1, 0, 0]
    n = int(round(1 * sfreq)) + 1
    filt = signal.firwin2(n, freq, gain, nyq=nyq)
    return filt

# Calculate features according to ratio bands
def get_bands_feats(df, sfreq=125.):

    electrodes = df.columns

    feats = {}

    for el in electrodes:
        freqs, psds = signal.welch(df[el], sfreq, nperseg=1024)
        fres = freqs[1] - freqs[0]
        psd_df = pd.DataFrame(data={'freqs': freqs, 'psds': psds})
    
        #feats[get_col_name('bands', 'alpha', el)] = simps(psd_df['psds'].loc[
        #    (psd_df['freqs'] >= band_bounds['alpha'][0]) &
        #    (psd_df['freqs'] <= band_bounds['alpha'][1])], dx=fres)

        feats[get_col_name('bands', 'alpha', el)] = psd_df.loc[
            (psd_df['freqs'] >= band_bounds['alpha'][0]) &
            (psd_df['freqs'] <= band_bounds['alpha'][1])]['psds'].sum()/psd_df['freqs'].sum()
        
        #feats[get_col_name('bands', 'beta', el)] = simps(psd_df['psds'].loc[
        #    (psd_df['freqs'] >= band_bounds['beta'][0]) &
        #    (psd_df['freqs'] <= band_bounds['beta'][1])], dx=fres)

        feats[get_col_name('bands', 'beta', el)] = psd_df.loc[
            (psd_df['freqs'] >= band_bounds['beta'][0]) &
            (psd_df['freqs'] <= band_bounds['beta'][1])]['psds'].sum()/psd_df['freqs'].sum()

        #feats[get_col_name('bands', 'theta', el)] = simps(psd_df['psds'].loc[
        #    (psd_df['freqs'] >= band_bounds['theta'][0]) &
        #    (psd_df['freqs'] <= band_bounds['theta'][1])], dx=fres)
        
        feats[get_col_name('bands', 'theta', el)] = psd_df.loc[
            (psd_df['freqs'] >= band_bounds['theta'][0]) &
            (psd_df['freqs'] <= band_bounds['theta'][1])]['psds'].sum()/psd_df['freqs'].sum()

        #feats[get_col_name('bands', 'gamma', el)] = simps(psd_df['psds'].loc[
        #    (psd_df['freqs'] >= band_bounds['gamma'][0]) &
        #    (psd_df['freqs'] <= band_bounds['gamma'][1])], dx=fres)
        
        feats[get_col_name('bands', 'gamma', el)] = psd_df.loc[
            (psd_df['freqs'] >= band_bounds['gamma'][0]) &
            (psd_df['freqs'] <= band_bounds['gamma'][1])]['psds'].sum()/psd_df['freqs'].sum()

    return feats

# Calculate spectral connectivity
def get_mne_spec_con_feats(df, sfreq=125., band=None, method='coh'):

    electrodes = df.columns
    res = spectral_connectivity(
        df[electrodes].values.T.reshape(1, len(electrodes), -1),
        method=method, sfreq=sfreq, verbose=False)

    data = res[0]
    freqs = res[1]

    def filter(arr):
        if band is None:
            return arr
        else:
            start_idx = np.where(freqs > band_bounds[band][0])[0][0]
            end_idx = np.where(freqs < band_bounds[band][1])[0][-1] + 1
            return arr[start_idx:end_idx]

    d = {}

    idx_electrodes_dict = {i: e for i, e in enumerate(electrodes)}

    for idx_1, idx_2 in itertools.combinations(range(len(electrodes)), 2):
        el_1 = idx_electrodes_dict[idx_1]
        el_2 = idx_electrodes_dict[idx_2]
        d[get_col_name(method, band, el_1, el_2)] = filter(data[idx_2, idx_1]).mean()

    return d

def get_envelope_feats(df, sfreq=125., band='alpha'):

    electrodes = df.columns

    df = df.copy()
    new_df = pd.DataFrame()
    if band is not None:
        filt = get_filter(sfreq, band)
    else:
        filt = None

    for el in electrodes:
        sig = df[el]
        if filt is not None:
            sig = np.convolve(filt, df[el], 'valid')
        sig = hilbert(sig)
        sig = np.abs(sig)
        new_df[el + '_env'] = sig

    d = {}

    idx_electrodes_dict = {i: e for i, e in enumerate(electrodes)}

    for idx_1, idx_2 in itertools.combinations(range(len(electrodes)), 2):
        el_1 = idx_electrodes_dict[idx_1]
        el_2 = idx_electrodes_dict[idx_2]
        series_1 = new_df[el_1 + '_env']
        series_2 = new_df[el_2 + '_env']
        d[get_col_name('env', band, el_1, el_2)] = pearsonr(series_1, series_2)[0]

    return d


def get_psi_feats(df, sfreq=125., band='alpha'):

    electrodes = df.columns

    df = df.copy()
    alpha_filter = get_filter(sfreq=sfreq, band=band)

    df = df[electrodes]
    for el in electrodes:
        df[el] = np.convolve(alpha_filter, df[el], 'same')

    vals = df.values
    vals = vals.transpose(1, 0)
    vals = vals[None, :, :]

    psi, freqs, times, n_epochs, _ = phase_slope_index(vals, sfreq=sfreq, verbose=False)
    d = {}
    for i in range(psi.shape[0]):
        for j in range(i):
            d[get_col_name('psi', band, electrodes[i], electrodes[j])] = psi[i, j, 0]
    return d

def calc_features(method_name, name_file, out_path):
    
    data_path = out_path

    methods = {
    'coh': partial(get_mne_spec_con_feats, band=None, method='coh'),
    'coh-alpha': partial(get_mne_spec_con_feats, band='alpha', method='coh'),
    'coh-beta': partial(get_mne_spec_con_feats, band='beta', method='coh'),
    'coh-theta': partial(get_mne_spec_con_feats, band='theta', method='coh'),
    'env': partial(get_envelope_feats, band=None),
    'env-alpha': partial(get_envelope_feats, band='alpha'),
    'env-beta': partial(get_envelope_feats, band='beta'),
    'env-theta': partial(get_envelope_feats, band='theta'),
    'bands': get_bands_feats,
    'psi': get_psi_feats,
    }

    f = methods[method_name]
    
    def unity_func(x):
        return x

    df_filter_func = unity_func
        
    #print('Started features stage -', method_name)

    path_file_path = join(data_path, '{}.csv'.format(name_file))
    path_df = pd.read_csv(path_file_path)
    # required columns check
    assert all([col in path_df.columns for col in ['fn', 'target']])

    features_path = get_feature_path(method_name, out_path)

    new_rows = []

    for i, row in tqdm(path_df.iterrows(), total=len(path_df)):
        try:
            path = join(data_path, row['fn'])
            df = pd.read_csv(path, index_col='time')
            df = df_filter_func(df)
            new_row = f(df)
        except AssertionError:
            print('Error in file ' + row['fn'])
            continue
        except FileNotFoundError:
            print('Not found - ' + row['fn'])
            continue

        for col in ['fn', 'target']:
            new_row[col] = row[col]
        new_rows.append(new_row)

        res_df = pd.DataFrame(new_rows)
        res_df.to_csv(features_path, index=False)
        
    #return res_df

### 1) Computing all features

In [4]:
import time

method_list=['coh','coh-alpha','coh-beta','coh-theta','env','env-alpha','env-beta','env-theta','bands','psi']

for m in method_list:
    print('Started features stage -', m)
    time.sleep(1)
    calc_features(m, 'processed', 'datasets') #method_name, name_file, out_path

Started features stage - coh


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 185/185 [04:35<00:00,  1.49s/it]


Started features stage - coh-alpha


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 185/185 [04:03<00:00,  1.32s/it]


Started features stage - coh-beta


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 185/185 [00:58<00:00,  3.14it/s]


Started features stage - bands


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 185/185 [00:43<00:00,  4.30it/s]


Started features stage - psi


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 185/185 [03:43<00:00,  1.21s/it]


In [5]:
m1 = pd.read_csv('datasets/coh.csv')
m1

,coh_nofilt_fp1_fp2,coh_nofilt_fp1_f7,coh_nofilt_fp1_f3,coh_nofilt_fp1_fz,coh_nofilt_fp1_f4,coh_nofilt_fp1_f8,coh_nofilt_fp1_t3,coh_nofilt_fp1_c3,coh_nofilt_fp1_c4,coh_nofilt_fp1_t4,...,coh_nofilt_pz_o1,coh_nofilt_pz_o2,coh_nofilt_p4_t6,coh_nofilt_p4_o1,coh_nofilt_p4_o2,coh_nofilt_t6_o1,coh_nofilt_t6_o2,coh_nofilt_o1_o2,fn,target
0,0.849397,0.877339,0.857365,0.864015,0.795604,0.715263,0.611386,0.674974,0.672999,0.609560,...,0.842346,0.719059,0.868253,0.785199,0.868845,0.699483,0.897227,0.783050,00b2d6e257e2f615.csv,trauma
1,0.682347,0.795053,0.775378,0.657702,0.641092,0.713337,0.647718,0.536908,0.380917,0.541196,...,0.761094,0.542005,0.485035,0.620447,0.485035,0.825348,1.000000,0.825348,09769097749fb286.csv,trauma
2,0.897231,0.847988,0.861601,0.828406,0.752612,0.744819,0.581250,0.791440,0.517047,0.518814,...,0.787946,0.672239,0.746679,0.704227,0.675528,0.841782,0.839497,0.839400,0b84dd748e7d5edd.csv,trauma
3,0.755840,0.844175,0.833709,0.886321,0.793390,0.741746,0.808783,0.861057,0.766067,0.809645,...,0.924519,0.741273,0.953717,0.835530,0.839065,0.853975,0.863793,0.825079,158ce5e17a662599.csv,trauma
4,0.639640,0.906704,0.887078,0.892046,0.588968,0.631242,0.710337,0.635790,0.448039,0.564691,...,0.670676,0.518230,0.768984,0.550258,0.739940,0.600236,0.782397,0.634265,17df70855fa4922a.csv,trauma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,0.396877,0.656498,0.576064,0.494556,0.420211,0.428643,0.563125,0.661773,0.500596,0.401992,...,0.519891,0.502449,0.463810,0.409775,0.623870,0.424660,0.406684,0.399532,b5abacb75ebc40b8.csv,healthy
181,0.462236,0.467286,0.384045,0.431943,0.400531,0.356827,0.378937,0.367412,0.380954,0.357517,...,0.760494,0.740467,0.848179,0.698284,0.788473,0.737689,0.874987,0.818709,bb47addde80c51c0.csv,healthy
182,0.833737,0.924463,0.834045,0.797401,0.716631,0.788221,0.774200,0.666065,0.550072,0.700391,...,0.811561,0.689844,0.766539,0.730606,0.842840,0.790732,0.811553,0.805467,bc9ecd77d29ef5fb.csv,healthy
183,0.611631,0.875651,0.906435,0.882049,0.470444,0.513214,0.730145,0.792010,0.396715,0.473648,...,0.880845,0.509720,0.820510,0.635592,0.796423,0.760664,0.849810,0.668075,bcfaa165d982034b.csv,healthy


In [6]:
m2 = pd.read_csv('datasets/bands.csv')
m2

,bands_alpha_fp1,bands_beta_fp1,bands_theta_fp1,bands_gamma_fp1,bands_alpha_fp2,bands_beta_fp2,bands_theta_fp2,bands_gamma_fp2,bands_alpha_f7,bands_beta_f7,...,bands_alpha_o1,bands_beta_o1,bands_theta_o1,bands_gamma_o1,bands_alpha_o2,bands_beta_o2,bands_theta_o2,bands_gamma_o2,fn,target
0,0.011429,0.006285,0.007146,0.000176,0.012048,0.006697,0.007730,0.000212,0.006844,0.005105,...,0.022065,0.004016,0.002469,0.000149,0.035880,0.005168,0.003988,0.000162,00b2d6e257e2f615.csv,trauma
1,0.114268,0.014217,0.013358,0.000265,0.116196,0.013530,0.007571,0.000280,0.104544,0.012776,...,0.066383,0.012111,0.010662,0.000159,0.089026,0.015053,0.010968,0.000271,09769097749fb286.csv,trauma
2,0.008311,0.007985,0.005004,0.000048,0.008255,0.008058,0.004632,0.000067,0.010385,0.009095,...,0.014043,0.008276,0.004910,0.000083,0.013040,0.007883,0.005657,0.000066,0b84dd748e7d5edd.csv,trauma
3,0.001594,0.008687,0.001981,0.000345,0.001259,0.003058,0.001975,0.000106,0.002884,0.024153,...,0.003806,0.009390,0.003869,0.000275,0.002508,0.006640,0.004404,0.000109,158ce5e17a662599.csv,trauma
4,0.065740,0.013531,0.008472,0.001411,0.050149,0.008920,0.005406,0.000698,0.053171,0.011563,...,0.132640,0.023853,0.015792,0.002290,0.172693,0.026239,0.016452,0.001640,17df70855fa4922a.csv,trauma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,0.033886,0.037463,0.083571,0.007562,0.090162,0.090434,0.268528,0.022914,0.021865,0.026020,...,0.063334,0.066810,0.164030,0.015815,0.044101,0.041745,0.131862,0.010972,b5abacb75ebc40b8.csv,healthy
181,0.000221,0.000314,0.000322,0.000061,0.000206,0.000241,0.000278,0.000052,0.001530,0.000920,...,0.016213,0.004947,0.005888,0.000581,0.014890,0.004814,0.005970,0.000530,bb47addde80c51c0.csv,healthy
182,0.003748,0.008026,0.005264,0.001029,0.002610,0.005732,0.004342,0.000843,0.003682,0.007597,...,0.011645,0.011645,0.005227,0.000880,0.006323,0.008482,0.005028,0.000774,bc9ecd77d29ef5fb.csv,healthy
183,0.023617,0.014221,0.014385,0.001164,0.006882,0.004902,0.006951,0.000454,0.020927,0.013584,...,0.044583,0.017070,0.015418,0.000867,0.039263,0.011665,0.009533,0.000356,bcfaa165d982034b.csv,healthy


### 2) Processing csv files:

In [7]:
path='datasets/'

meth_names=['coh','coh_alpha','coh_beta','coh_theta','env','env_alpha','env_beta','env_theta','bands','psi']
lst_files=[]

for l in meth_names:
  lst_files.append(pd.read_csv(path+'{}.csv'.format(l)))

df_features = pd.concat(lst_files, axis=1).T.drop_duplicates().T

In [8]:
df_features

,coh_nofilt_fp1_fp2,coh_nofilt_fp1_f7,coh_nofilt_fp1_f3,coh_nofilt_fp1_fz,coh_nofilt_fp1_f4,coh_nofilt_fp1_f8,coh_nofilt_fp1_t3,coh_nofilt_fp1_c3,coh_nofilt_fp1_c4,coh_nofilt_fp1_t4,...,psi_alpha_o2_t3,psi_alpha_o2_c3,psi_alpha_o2_c4,psi_alpha_o2_t4,psi_alpha_o2_t5,psi_alpha_o2_p3,psi_alpha_o2_pz,psi_alpha_o2_p4,psi_alpha_o2_t6,psi_alpha_o2_o1
0,0.849397,0.877339,0.857365,0.864015,0.795604,0.715263,0.611386,0.674974,0.672999,0.60956,...,1.13733,-2.53551,2.85189,-2.41336,-7.88625,-3.43036,-2.94028,4.75534,-2.37289,-6.15134
1,0.682347,0.795053,0.775378,0.657702,0.641092,0.713337,0.647718,0.536908,0.380917,0.541196,...,-0.0696984,-0.223584,2.8267,-5.64508,-0.0232415,-0.92827,1.02514,2.12541,0,-0.129469
2,0.897231,0.847988,0.861601,0.828406,0.752612,0.744819,0.58125,0.79144,0.517047,0.518814,...,5.71915,4.84181,-4.84346,-2.76454,-1.0467,-2.97276,-6.5695,-2.92858,-0.4435,-0.663475
3,0.75584,0.844175,0.833709,0.886321,0.79339,0.741746,0.808783,0.861057,0.766067,0.809645,...,1.91029,0.0645797,1.45091,-1.03404,-0.911907,-1.69319,-1.26354,-3.62276,-2.16622,1.22638
4,0.63964,0.906704,0.887078,0.892046,0.588968,0.631242,0.710337,0.63579,0.448039,0.564691,...,2.34378,-0.0195181,-0.481569,1.60823,0.394639,0.247216,0.442689,0.863781,-0.00693753,-0.153289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,0.396877,0.656498,0.576064,0.494556,0.420211,0.428643,0.563125,0.661773,0.500596,0.401992,...,-134.552,-148.02,-174.643,-128.885,-141.926,-110.661,-153.217,-11.5022,-156.377,-173.192
181,0.462236,0.467286,0.384045,0.431943,0.400531,0.356827,0.378937,0.367412,0.380954,0.357517,...,-15.9811,-18.8163,-18.1835,-11.6018,-7.54197,-16.9358,-17.9196,-14.4301,-6.42658,-7.38829
182,0.833737,0.924463,0.834045,0.797401,0.716631,0.788221,0.7742,0.666065,0.550072,0.700391,...,5.92326,7.67692,19.28,37.2097,3.5576,3.77069,4.75503,0.985402,3.50039,0.911847
183,0.611631,0.875651,0.906435,0.882049,0.470444,0.513214,0.730145,0.79201,0.396715,0.473648,...,-26.7851,51.3452,-32.4538,-37.7736,70.6245,-18.839,-11.2204,-21.2441,-10.816,-16.0599


In [9]:
df_features.to_csv('mldataset/allfeatures.csv', index=False)